In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
import pandas as pd
import random

import itertools
import time

/Users/waizwafiq/Documents/FSKTM/Semester 6/WIX3001 Soft Computing/wix3001_assignment/wix3001_assignment/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class ConvNet(nn.Module):
    def __init__(self, input_channels, hidden_channels, num_classes, kernel_sizes=None):
        super(ConvNet, self).__init__()

        if kernel_sizes is None:
            k = [[3]]
        else:
            k = kernel_sizes

        self.conv1 = nn.Conv2d(input_channels, hidden_channels[0], kernel_size=k[0][0], stride=1, padding=1)
        self.relu1 = nn.ReLU()

        self.conv_layers = nn.ModuleList()
        self.pool_layers = nn.ModuleList()  # Add module list for pooling layers

        for i in range(1, len(hidden_channels)):
            self.conv_layers.append(nn.Conv2d(hidden_channels[i-1], hidden_channels[i], kernel_size=k[i][0], stride=1, padding=1))
            self.relu = nn.ReLU()
            self.pool_layers.append(nn.MaxPool2d(kernel_size=k[i][1], stride=2))  # Add max pooling layer

        self.fc = nn.Linear(hidden_channels[-1], num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)

        for conv_layer, pool_layer in zip(self.conv_layers, self.pool_layers):  # Iterate over conv and pool layers
            out = conv_layer(out)
            out = self.relu(out)
            out = pool_layer(out)  # Apply max pooling

        out = F.avg_pool2d(out, kernel_size=out.size()[2:])  # Global average pooling
        out = out.view(out.size(0), -1)  # Flatten the tensor

        out = self.fc(out)

        if not self.training:
            out = F.softmax(out, dim=1)
        
        return out

In [32]:
def trainModel(model, data_loader, epochSize=20):
    train_loader, test_loader = data_loader[0], data_loader[1]

    loss_fn = nn.CrossEntropyLoss()
    opt = torch.optim.Adam(model.parameters())

    loss_per_epoch = []
    train_acc_per_epoch = []
    test_acc_per_epoch = []
    total_acc_per_epoch = []
    time_per_epoch = []
    exec_time = []

    start_total_time = time.time()
    for epoch in range(epochSize):

        loss = 0
        start_epoch_time = time.time()

        count = 1
        for input_batch, target_batch in train_loader:
            print(
                f"Epoch: {epoch + 1} => {time.time() - start_epoch_time:.2f}s {(count/count_batch_train)*100:.3f}%", end='')

            # Zero the gradients
            opt.zero_grad()

            # Forward pass
            predict_batch = model(input_batch)

            # Compute loss
            loss_batch = loss_fn(predict_batch, target_batch)

            # Backward pass and update weights
            loss_batch.backward()
            opt.step()

            loss += loss_batch.item()  # store the loss
            count += 1
            print('\r', end='', flush=True)

        loss_per_epoch.append(loss)
        # print(loss)

        # CALCULATE TRAIN ACCURACY
        correct = 0
        total = 0
        train_accuracy = 0

        with torch.no_grad():
            count = 1
            for images, labels in train_loader:
                print(f"Epoch: {epoch + 1} => {time.time() - start_epoch_time:.2f}s || Calculating Training Accuracy... {(count/count_batch_train)*100:.3f}%", end='', flush=True)
                # Forward pass
                outputs = model(images)

                # Get the predicted labels
                _, predicted = torch.max(outputs.data, 1)

                # Update counts
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                count += 1
                print('\r', end='', flush=True)

        # Calculate accuracy
        train_accuracy = correct / total
        train_acc_per_epoch.append(train_accuracy)

        # CALCULATE TEST ACCURACY
        correct = 0
        total = 0
        test_accuracy = 0

        with torch.no_grad():
            count = 1
            for images, labels in test_loader:
                print(f"Epoch: {epoch + 1} => {time.time() - start_epoch_time:.2f}s || Calculating Testing Accuracy... {(count/count_batch_test)*100:.3f}%", end='', flush=True)
                # Forward pass
                outputs = model(images)

                # Get the predicted labels
                _, predicted = torch.max(outputs.data, 1)

                # Update counts
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                count += 1
                print('\r', end='', flush=True)

        # Calculate accuracy
        test_accuracy = correct / total
        test_acc_per_epoch.append(test_accuracy)

        time_epoch = time.time() - start_epoch_time
        time_current = time.time() - start_total_time

        time_per_epoch.append(time_epoch)
        exec_time.append(time_current)

        total_accuracy = 0.3*train_accuracy + 0.7*test_accuracy
        total_acc_per_epoch.append(total_accuracy)

        print(f'Epoch: {epoch+1} || Loss: {loss} || Train Acc: {train_accuracy * 100:.4f}% || Test Acc: {test_accuracy * 100:.4f}% || Total Acc: {total_accuracy * 100:.4f}% || Epoch Time: {time_epoch:.4f} s || Current Runtime: {time_current:.4f} s')

    output = {
        'loss': loss_per_epoch,
        'train_acc': train_acc_per_epoch,
        'test_acc': test_acc_per_epoch,
        'total_acc': total_acc_per_epoch,
        'epoch_time': time_per_epoch,
        'exec_time': exec_time
    }

    return output

In [148]:
def fitness(individual, dataloader):
    # Create ConvNet instance with the provided individual configuration
    try:
        h = individual[0]
        k_0 = [[2]]
        k_n = k_0 + individual[1]
    except Exception:
        h = individual[0]
        h = [h]
        k_0 = [[2]]
        k_n = k_0 + [individual[2:]]
        # print(h, k_n)

    # print(individual)
    model = ConvNet(1, hidden_channels=h, num_classes=5, kernel_sizes=k_n)
    result = trainModel(model, dataloader, epochSize=10)
    return result['total_acc'][-1]

def define_fitness(population, dataloader):
    fitness_per_individual = []
    for individual in population:
        fitness_per_individual.append(fitness(individual, dataloader))

    return fitness_per_individual

def generate_population(population_size, layers, hidden_channels_range, kernel_size_range):
    np.random.seed(4)
    population = []
    for _ in range(population_size):
        # For each population,
        chromosome_h = []
        for _ in range(layers):
            # create chromosome for the number of hidden channels per layer e.g: (layers = 3) ->[256, 128, 64]
            filters = np.random.randint(
                hidden_channels_range[0], hidden_channels_range[1]+1
            )
            chromosome_h.append(filters)

        chromosome_k = []
        for _ in range(layers):
            # create chromosome for the kernel sizes per layer (2D list, each row represents layer)
            kernel_size = np.random.randint(
                kernel_size_range[0], kernel_size_range[1]+1, (1, 2)
            )
            chromosome_k.append(list(kernel_size[0]))
        population.append((chromosome_h, chromosome_k))

    return population

def DataFrame_Pop(pop_unstructured):
    population = []
    for i in range(len(pop_unstructured)):
        lst1 = np.array(pop_unstructured[i][0])
        lst2 = np.array(pop_unstructured[i][1])
        flattened = np.concatenate(([i+1], lst1, lst2.flatten()))

        population.append(flattened.tolist())
    col_name1 = [f"conv_h (Layer {i+1})" for i in range(len(pop_unstructured[0][0]))]
    col_name2 = [f"conv_k (Layer {i+1})" for i in range(len(pop_unstructured[0][0]))]
    col_name3 = [f"pool_k (Layer {i+1})" for i in range(len(pop_unstructured[0][0]))]
    col_name4 = list(itertools.chain(*zip(col_name2, col_name3)))

    cols = ['Individual'] + col_name1 + col_name4
    return pd.DataFrame(population, columns=cols, index=None)

In [6]:
# Define the directory path
data_dir = './processed_dataset'

# Create the ImageFolder dataset
dataset = datasets.DatasetFolder(data_dir, loader=torch.load, extensions=".pt")

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # Remaining 20% for testing

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Set DataLoader
batchSize = 16  # Rule of thumb is to set to the power of 2. In this case 2^7
train_loader = DataLoader(train_dataset, batch_size=batchSize,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False) # no need to shuffle test data

count_batch_train, count_batch_test = 0, 0
for xb, yb in train_loader:
  print(count_batch_train, end='', flush=True)
  count_batch_train += 1
  print("\r", end='', flush=True)
print(f'There are {count_batch_train} batches in train_loader')

for xb, yb in test_loader:
  print(count_batch_test, end='', flush=True)
  count_batch_test += 1
  print("\r", end='', flush=True)
print(f'There are {count_batch_test} batches in test_loader')

for i, j in train_loader:
    size = i.shape
    break

print(size)

There are 413 batches in train_loader
There are 104 batches in test_loader
torch.Size([16, 1, 224, 224])


In [63]:
pop = generate_population(5, 1, [1, 4], [1, 2])
df = DataFrame_Pop(pop)

In [60]:
fitness_scores = define_fitness(pop, [train_loader, test_loader])

([3], [[1, 2]])
Epoch: 1 || Loss: 669.7146828174591 || Train Acc: 21.3485% || Test Acc: 20.3636% || Total Acc: 20.6591% || Epoch Time: 56.2944 s || Current Runtime: 56.2944 s
Epoch: 2 || Loss: 647.6352045536041 || Train Acc: 27.3939% || Test Acc: 27.5758% || Total Acc: 27.5212% || Epoch Time: 50.6324 s || Current Runtime: 106.9271 s
Epoch: 3 || Loss: 622.2057185173035 || Train Acc: 37.8333% || Test Acc: 39.6970% || Total Acc: 39.1379% || Epoch Time: 68.2943 s || Current Runtime: 175.2216 s
Epoch: 4 || Loss: 591.2863721847534 || Train Acc: 37.7424% || Test Acc: 39.5758% || Total Acc: 39.0258% || Epoch Time: 61.6021 s || Current Runtime: 236.8241 s
Epoch: 5 || Loss: 559.4222121238708 || Train Acc: 42.8333% || Test Acc: 43.6364% || Total Acc: 43.3955% || Epoch Time: 63.4569 s || Current Runtime: 300.2813 s
Epoch: 6 || Loss: 532.1628519296646 || Train Acc: 43.6667% || Test Acc: 45.3939% || Total Acc: 44.8758% || Epoch Time: 64.1225 s || Current Runtime: 364.4041 s
Epoch: 7 || Loss: 510.562

In [61]:
fitness_scores

[0.4642272727272727,
 0.42456060606060597,
 0.4629545454545454,
 0.4435,
 0.4939545454545454]

In [66]:
df['Fitness'] = fitness_scores
df

,Individual,conv_h (Layer 1),conv_k (Layer 1),pool_k (Layer 1),Fitness
0,1,3,1,2,0.464227
1,2,2,2,1,0.424561
2,3,4,1,1,0.462955
3,4,2,1,1,0.443500
4,5,4,2,1,0.493955


In [151]:
def crossover(df_pop, dataloader):
    df_pop_sorted = df_pop.sort_values('Fitness', ascending=False) # Sort the rows by the fitness score in descending order
    c = len(df_pop.columns)
    r = len(df_pop)
    n_layers = int((c - 2)/3) # n = (c - 2)/3

    children = []
    for i in range(r-1):
        parents = df_pop_sorted.iloc[i:i+2] # Get top 2
        parent1 = parents.iloc[0, 1:c-1].to_list()
        parent2 = parents.iloc[1, 1:c-1].to_list()
        offspring = parent1[:n_layers] + parent2[n_layers:n_layers+1] + parent2[n_layers+1:]

        offspring = [int(x) for x in offspring]

        children.append([r + i + 1] + offspring)
    
    df_children = pd.DataFrame(children, columns=df_pop.columns[:-1])
    fitness_scores = []
    for child in children:
        fitness_scores.append(fitness(child, dataloader))

    df_children['Fitness'] = fitness_scores

    return df_children

df_children = crossover(df, [train_loader, test_loader])
    

Epoch: 1 || Loss: 665.4075813293457 || Train Acc: 20.8939% || Test Acc: 21.5758% || Total Acc: 21.3712% || Epoch Time: 38.9977 s || Current Runtime: 38.9977 s
Epoch: 2 || Loss: 662.7623431682587 || Train Acc: 30.0606% || Test Acc: 28.7879% || Total Acc: 29.1697% || Epoch Time: 38.3263 s || Current Runtime: 77.3241 s
Epoch: 3 || Loss: 658.5742242336273 || Train Acc: 35.2121% || Test Acc: 33.0909% || Total Acc: 33.7273% || Epoch Time: 38.3826 s || Current Runtime: 115.7067 s
Epoch: 4 || Loss: 646.6983791589737 || Train Acc: 39.3939% || Test Acc: 39.0909% || Total Acc: 39.1818% || Epoch Time: 38.2294 s || Current Runtime: 153.9361 s
Epoch: 5 || Loss: 622.806077837944 || Train Acc: 37.5909% || Test Acc: 38.7273% || Total Acc: 38.3864% || Epoch Time: 38.6196 s || Current Runtime: 192.5558 s
Epoch: 6 || Loss: 591.8522716760635 || Train Acc: 40.2727% || Test Acc: 40.7273% || Total Acc: 40.5909% || Epoch Time: 38.5138 s || Current Runtime: 231.0696 s
Epoch: 7 || Loss: 561.4208903312683 || Trai

In [154]:
pd.concat([df, df_children])

,Individual,conv_h (Layer 1),conv_k (Layer 1),pool_k (Layer 1),Fitness
0,1,3,1,2,0.464227
1,2,2,2,1,0.424561
2,3,4,1,1,0.462955
3,4,2,1,1,0.443500
4,5,4,2,1,0.493955
0,6,4,1,2,0.435485
1,7,3,1,1,0.486030
2,8,4,1,1,0.478364
3,9,2,2,1,0.472894
